In [1]:
import tensorflow as tf
import numpy as np
import csv
from tqdm import tqdm
import pandas as pd
import random
import pickle

root = './data/'

In [2]:
global false, null, true
false =False
true=True
null=''

In [3]:
with open(root+'Magazine_5.json') as fin:
    df = {}
    for i, line in enumerate(fin):
        df[i] = eval(line)
    reviews_df = pd.DataFrame.from_dict(df, orient='index')

In [4]:
with open(root+'np_prepro/reviews.pkl', 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL)

In [5]:
with open(root+'meta_Magazine.json') as fin:
    df = {}
    for i, line in enumerate(fin):
        df[i] = eval(line)
    meta_df = pd.DataFrame.from_dict(df, orient='index')

meta_df = meta_df[meta_df['asin'].isin(reviews_df['asin'].unique())]
meta_df = meta_df.reset_index(drop=True)
with open(root+'np_prepro/meta.pkl', 'wb') as f:
    pickle.dump(meta_df, f, pickle.HIGHEST_PROTOCOL)

In [6]:
reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
meta_df = meta_df[['asin', 'category']]
# only one category...
meta_df['category'] = meta_df['category'].map(lambda x: x[-1] if x else 'Video_Game')

In [7]:
reviews_df

,reviewerID,asin,unixReviewTime
0,A5QQOOZJOVPSF,B00005N7P0,1393372800
1,A5RHZE7B8SV5Q,B00005N7PS,1078531200
2,A1RPTVW5VEOSI,B00005N7PS,1058227200
3,A1SFRBCMW8XVBW,B00005N7PS,1422662400
4,A1IU9VPCBKZPE8,B00005N7P0,1286236800
...,...,...,...
2370,A18X3E6V8DGIDZ,B00X6LREJU,1517961600
2371,A1Y98LVYJ0YZJ0,B00X6LREHM,1488499200
2372,A1VTKYW3YQEHTN,B00X6LREHM,1484438400
2373,ASU7EOBD3Y4BV,B01HI8V10E,1496102400


In [8]:
meta_df

,asin,category
0,B00005N7P0,Computers &amp; Internet
1,B00005N7QG,Recipes & Techniques
2,B00005N7Q5,Hunting & Firearms
3,B00005N7PN,Design & Decoration
4,B00005N7PG,"Travel, City & Regional"
...,...,...
244,B00X6LREH2,Video_Game
245,B00X6LREJU,Video_Game
246,B00X6LREHM,"Cooking, Food & Wine"
247,B01CF3ECNK,Recipes & Techniques


In [9]:
def build_map(df, col_name):
    key = sorted(df[col_name].unique().tolist())
    m = dict(zip(key, range(len(key))))
    df[col_name] = df[col_name].map(lambda x: m[x])
    return m, key

In [10]:
asin_map, asin_key = build_map(meta_df, 'asin')
cate_map, cate_key = build_map(meta_df, 'category')
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

In [11]:
user_count, item_count, cate_count, example_count =\
    len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]
print('user_count: %d\titem_count: %d\tcate_count: %d\texample_count: %d' %
      (user_count, item_count, cate_count, example_count))

user_count: 348	item_count: 151	cate_count: 71	example_count: 2375


In [12]:
meta_df = meta_df.sort_values('asin')
meta_df = meta_df.reset_index(drop=True)

In [13]:
reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x] if x in asin_map else 0)
reviews_df = reviews_df.sort_values(['reviewerID', 'unixReviewTime'])
reviews_df = reviews_df.reset_index(drop=True)

In [14]:
cate_list = [meta_df['category'][i] for i in range(len(asin_map))]
cate_list = np.array(cate_list, dtype=np.int32)

In [15]:
with open(root+'np_prepro/remap.pkl', 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL) # uid, iid
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL) # cid of iid line
    pickle.dump((user_count, item_count, cate_count, example_count),
              f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((asin_key, cate_key, revi_key), f, pickle.HIGHEST_PROTOCOL)

In [16]:
random.seed(1234)

train_set = []
test_set = []
for reviewerID, hist in reviews_df.groupby('reviewerID'):
    pos_list = hist['asin'].tolist()
    neg_list = []
    for _ in range(len(pos_list)):
        neg = pos_list[0]
        while neg in pos_list + neg_list :
            neg = random.randint(0, item_count-1)
        neg_list.append(neg)
        
    for i in range(1, len(pos_list)-1):
        hist = pos_list[:i]
        train_set.append((reviewerID, hist, pos_list[i], 1))
        train_set.append((reviewerID, hist, neg_list[i], 0))
    label = (pos_list[-1], neg_list[-1])
    test_set.append((reviewerID, hist, label))

random.shuffle(train_set)
random.shuffle(test_set)

assert len(test_set) == user_count

with open(root+'np_prepro/dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)